In [1]:
test_folder_path="/home/fate/covid19_CT/input/test"

In [2]:
import os
import gc
import cv2
import math
import copy
import time
import random
import shutil, sys    

import glob
# For data manipulation
import numpy as np
import pandas as pd

# Pytorch Imports
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader
from torch.cuda import amp

# Albumentations for augmentations
import albumentations as A
from albumentations.pytorch import ToTensorV2


from sklearn.metrics import f1_score,roc_auc_score


import timm
from timm.models.efficientnet import *

# Utils
import joblib
from tqdm import tqdm
from collections import defaultdict


import warnings
warnings.filterwarnings("ignore")

from sklearn.metrics import f1_score

import matplotlib.pyplot as plt
from scipy import ndimage

In [3]:
test_ct_list=list(glob.glob(os.path.join(test_folder_path, "*"))) 

In [4]:
len(test_ct_list)

5281

In [5]:
# test_ct_list=test_ct_list[-400:-390]   #quick test1
# test_ct_list=test_ct_list[-1] #quick test2

In [6]:
#quick test2
if isinstance(test_ct_list, str):
    test_ct_list=[test_ct_list]
    print("run this code")

In [7]:
def autocropmin(image, threshold=100, kernsel_size = 10):
        
    img = image.copy()
    
    SIZE = img.shape[0]
    imgfilt = ndimage.minimum_filter(img, size=kernsel_size)
    img_b=np.where(imgfilt<threshold,0,255)
    a=img_b[:,:,0].sum(axis=1)
    a=np.concatenate(([0],a,[0]))

    a_=np.where(a==0)[0]
    mina=a_[np.argmax(a_[1:]-a_[:-1])]
    maxa=a_[np.argmax(a_[1:]-a_[:-1])+1]-1

    b=img_b[:,:,0].sum(axis=0)
    b=np.concatenate(([0],b,[0]))

    b_=np.where(b==0)[0]
    minb=b_[np.argmax(b_[1:]-b_[:-1])]
    maxb=b_[np.argmax(b_[1:]-b_[:-1])+1]-1

    if  mina!=maxa and minb!=maxb:
        imageout=img[mina:maxa,minb:maxb,:]
    else:
        imageout=img

    return imageout

In [8]:
diff_shape_ct_list=[]
for i in tqdm(range(len(test_ct_list))):
    
    tmp_list=list(glob.glob(os.path.join(test_ct_list[i], "*")))
    
    
    
    tmp_shape_set=set()
    for j in range(len(tmp_list)):
    
        str1=tmp_list[j]
        img=cv2.imread(str1)
        
        try:
            tmp_shape_set.add(img.shape)
        except:
            print("bug file")
            continue
        
        img=autocropmin(img)


        str1=str1.replace(test_folder_path,"work_test/test_crop")


        folder_path="/".join(str1.split("/")[:-1])
        
        if len(tmp_shape_set)!=1:
            shutil.rmtree(folder_path)
            diff_shape_ct_list.append(folder_path.split("/")[-1])
            print(folder_path)
            break
            
        if not os.path.exists(folder_path):
            os.makedirs(folder_path)
        cv2.imwrite(str1,img)


 23%|██▎       | 1221/5281 [1:03:10<2:40:02,  2.37s/it]

work_test/test_crop/ct_scan_4163


 27%|██▋       | 1426/5281 [1:13:05<3:08:03,  2.93s/it]

work_test/test_crop/ct_scan_165


 27%|██▋       | 1443/5281 [1:13:56<4:18:43,  4.04s/it]

...

 83%|████████▎ | 4385/5281 [3:37:50<59:24,  3.98s/it]  

bug file


 93%|█████████▎| 4886/5281 [4:01:20<12:36,  1.92s/it]  

work_test/test_crop/ct_scan_1694


100%|██████████| 5281/5281 [4:20:16<00:00,  2.96s/it]


In [9]:
for ct in diff_shape_ct_list:
    str1=test_folder_path+"/"+ct
    str2="work_test/test_crop/"+ct
    
    tmp_list=list(glob.glob(os.path.join(str1, "*")))
    last_file=str(len(tmp_list)-1)+".jpg"
    str1=str1+"/"+last_file
    str2=str2+"/"+last_file
    img=cv2.imread(str1)
    img=autocropmin(img)
    folder_path="/".join(str2.split("/")[:-1])
    
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    cv2.imwrite(str2,img)
    